In [1]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.12

In [3]:
!kaggle competitions download -c tensorflow-great-barrier-reef

100% 14.2G/14.2G [05:33<00:00, 44.5MB/s]
100% 14.2G/14.2G [05:33<00:00, 45.6MB/s]


In [ ]:
!unzip /content/tensorflow-great-barrier-reef.zip -d /content/data

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
!cd yolov5
!pip install -r /content/yolov5/requirements.txt  # install

In [8]:
import pandas as pd
from IPython.display import display
import numpy
import cv2
import matplotlib.pyplot as plt

In [54]:
train = pd.read_csv("/content/data/train.csv", index_col="image_id")
train.head()

,video_id,sequence,video_frame,sequence_frame,annotations
image_id,,,,,
0-0,0,40258,0,0,[]
0-1,0,40258,1,1,[]
0-2,0,40258,2,2,[]
0-3,0,40258,3,3,[]
0-4,0,40258,4,4,[]


In [ ]:
img = cv2.imread("/content/data/train_images/video_0/978.jpg")
from google.colab.patches import cv2_imshow

start_pt = (825, 224)
end_pt = (875, 259)
color = (0, 0, 255)
thickness = 2
image = cv2.rectangle(img, start_pt, end_pt, color, thickness)
cv2_imshow(image)
image.shape

In [65]:
import os
def txt_data(path):
  for folder in os.listdir(path):
    path_to_folder = os.path.join(path, folder)
    _, video_id = folder.split("_")
    
    for image in os.listdir(path_to_folder):
      path_to_image = os.path.join(path_to_folder, image)
      
      image_id, _ = image.split(".")
      image_match = video_id + "-" + image_id
      
      img_temp = cv2.imread(path_to_image)
      
      height, width, _ = img_temp.shape
      image_data = train.loc[image_match]
      annotation_list = image_data.annotations
      print(annotation_list)

      
      path_to_file = path_to_folder + "/" + image_id + ".txt"
      print(path_to_file)
      file_ = open(path_to_file, "w")
      if bool(annotation_list):
        file_.close()
        continue
      else:
        for i in annotation_list:
          
      break
    break

txt_data("/content/data/train_images")

[]
/content/data/train_images/video_1/3979.txt


In [63]:
print(bool([]))

False
